In [3]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None 
import datetime as dt
import matplotlib.pyplot as plt
import time
import csv
from pyproj import CRS, Transformer
import os
import rasterio
import geopandas as gpd
from shapely.geometry import Point
from rasterio.mask import mask
from rasterio.merge import merge
from shapely.geometry import box, Point
from pyproj import CRS, Transformer

# The following code is used to preform the economic analysis of speed limit change

In [5]:
#models = pd.read_csv(f"AAA_All_Vehicles_All_Model_160.csv")
print(models.columns)

# Create a list of the initial columns to include in the new DataFrame
initial_columns = [
    'VehicleName', 'VehicleClass', 'StartDate', 'EndDate', 
    'FuelObs', 'TrueTime', 'DistanceTravelled', 'FuelModel', 
    'Num_GPS_IGN_ON'
]

# Create the new DataFrame with selected columns
new_df = models[initial_columns].copy()

# Iterate through the columns in the original DataFrame
for col in models.columns:
    if col.startswith('ModelTime_'):
        # Create a new column: ((ModelTime_ - TrueTime) / DistanceTravelled)
        new_df[f'Change{col}_per_km'] = (models[col] - models['TrueTime']) / models['DistanceTravelled']
    elif col.startswith('FuelModel_'):
        # Create a new column: ((FuelModel_ - FuelModel) / DistanceTravelled)
        new_df[f'Change{col}_per_km'] = (models[col] - models['FuelModel']) / models['DistanceTravelled']
    elif col.startswith(('Fatal', 'Serious', 'Minor', 'Non-injury')):
        # Add the column with 1 subtracted from its values
        new_df[f'Diff{col}'] = models[col] - 1

# Display the new DataFrame
#new_df.to_csv(f"Difference_per_km_All_Vehicles_All_Models_160.csv")
print(new_df)

NameError: name 'models' is not defined

In [ ]:
#all_vehicles = pd.read_csv(f"Difference_per_km_All_Vehicles_All_Models_160.csv")
print(all_vehicles)

In [ ]:
#all_vehicles = pd.read_csv(f"Difference_per_km_All_Vehicles_All_Models_160.csv")
print(all_vehicles.loc[:, 'StartDate'].iloc[-1])
from datetime import datetime
import pandas as pd

# Function to handle multiple datetime formats
def parse_dates(date_series):
    formats = ['%Y-%m-%d %H:%M:%S', '%d/%m/%Y %H:%M']  # List of possible formats
    
    def try_parse(date):
        for fmt in formats:
            try:
                return datetime.strptime(date, fmt)
            except ValueError:
                continue
        raise ValueError(f"Unknown date format: {date}")

    return date_series.apply(try_parse)

# Apply the function to 'StartDate' and 'EndDate' columns
all_vehicles['StartDate'] = parse_dates(all_vehicles['StartDate'])
all_vehicles['EndDate'] = parse_dates(all_vehicles['EndDate'])


all_vehicles['StartDate'] = pd.to_datetime(all_vehicles['StartDate'])
all_vehicles['EndDate'] = pd.to_datetime(all_vehicles['EndDate'])
all_vehicles['TrueTime'] = (all_vehicles['EndDate']-all_vehicles['StartDate']).dt.total_seconds()
all_vehicles['Days'] = all_vehicles['TrueTime']/86400
print(all_vehicles)

In [ ]:
# Cost of Diesel in Blenhiem in 2020 using $2.189 per liter atm (this is current prices) 
# GPS data ranges from 1/5/2020 - 15/9/2020
Diesel_cost = 2.189

# Cost of Time Savings
payrate_per_hour = 100 #150???
payrate_per_seconds = payrate_per_hour / (60*60)

# Fatal, Serious, Minor, Non-injury number of crashes in 2020 and probability per day
num_crashes = [304, 2080,8483, 86939]
num_vehicles = 4712629
num_days_2020 = 365
cost_crashes = [16095200,890500,105400,3800]
prob_crashes_per_day = [x/(num_vehicles * num_days_2020) for x in num_crashes]
cost_crashes_per_day = [x*y for x,y in zip(cost_crashes, prob_crashes_per_day)]
print(prob_crashes_per_day)
print(cost_crashes_per_day)

In [ ]:
print(all_vehicles.columns)
print(all_vehicles.dtypes)
print(all_vehicles)

In [ ]:
df = all_vehicles
all_vehicles_cost_change = df[df.columns[~df.columns.str.startswith(("ChangeTime", "ChangeFuelModel")) |df.columns.str.endswith("_per_km")]].copy()

#Diesel_cost = 1.5
#cost_crashes_per_day = [0.2, 0.3, 0.4, 0.5]  # Example list

# Multiply columns starting with ChangeFuelModel by Diesel_cost
fuel_columns = all_vehicles_cost_change.columns[all_vehicles_cost_change.columns.str.startswith("ChangeFuelModel")]
all_vehicles_cost_change[fuel_columns] *= Diesel_cost

# Multiply columns starting with ChangeTime by payrate_per_seconds
time_columns = all_vehicles_cost_change.columns[all_vehicles_cost_change.columns.str.startswith("ChangeModelTime")]
all_vehicles_cost_change[time_columns] *=  payrate_per_seconds

# Multiply Diff* columns by Days and respective cost_crashes_per_day index
diff_prefixes = ("DiffFatal", "DiffSerious", "DiffMinor", "DiffNon-injury")
for i, prefix in enumerate(diff_prefixes):
    diff_columns = all_vehicles_cost_change.columns[all_vehicles_cost_change.columns.str.startswith(prefix)]
    print(all_vehicles_cost_change[diff_columns].columns)
    
    all_vehicles_cost_change[diff_columns] = all_vehicles_cost_change[diff_columns].mul(cost_crashes_per_day[i], axis=0)
    #all_vehicles_cost_change["Days"] * do not multiply by the value of days to have diff in terms of cost per day
    print(cost_crashes_per_day[i])
    print(prefix)
    print(all_vehicles_cost_change[diff_columns])

# Rename columns that were modified to include Cost_* at the start
modified_columns = list(fuel_columns) + list(time_columns)+ [
    col for prefix in diff_prefixes for col in all_vehicles_cost_change.columns if col.startswith(prefix)
]
all_vehicles_cost_change.rename(
    columns={col: f"Cost_{col}" for col in modified_columns},
    inplace=True,
)

Economic_costs = all_vehicles_cost_change

#all_vehicles_cost_change.to_csv(f"Cost_per_km_All_Vehicles_All_Models_160.csv")

In [ ]:
print(Economic_costs.columns)
print(Economic_costs.dtypes)
print(Economic_costs)

In [ ]:
# Select all columns that start with 'Cost_'
cost_columns = Economic_costs.columns[Economic_costs.columns.str.startswith('Cost_')]

# Multiply the values in the 'Cost_' columns by the 'Days' column in the same row
Economic_costs[cost_columns] = Economic_costs[cost_columns].mul(Economic_costs["DistanceTravelled"], axis=0)

Economic_costs.columns = Economic_costs.columns.str.replace('_per_km$', '', regex=True)

Eccony = Economic_costs.copy()
print(Eccony)

#Eccony.to_csv(f"Cost_per_FuelUp_All_Vehicles_All_Models_160.csv")

In [ ]:
#almost2 = pd.read_csv(f"Cost_per_km_All_Vehicles_All_Models_160.csv")
print(almost2.columns)
almost = almost2.copy()
print(almost)

In [ ]:
# Define the columns to exclude
exclude_columns = ['VehicleName', 'VehicleClass', 'StartDate', 'EndDate']

# Select columns for aggregation (all columns except the ones to exclude)
columns_for_aggregation = [col for col in almost.columns if col not in exclude_columns]

# First, group by VehicleName and calculate mean and std for other columns
vehicle_name_stats = almost.groupby('VehicleName')[columns_for_aggregation].agg(['mean', 'std']).reset_index()

# Add an 'Identifier' column for VehicleName
vehicle_name_stats['Identifier'] = vehicle_name_stats['VehicleName']
vehicle_name_stats = vehicle_name_stats.drop(columns=['VehicleName'])

# Now, group by VehicleClass and calculate mean and std for other columns
vehicle_class_stats = almost.groupby('VehicleClass')[columns_for_aggregation].agg(['mean', 'std']).reset_index()

# Add an 'Identifier' column for VehicleClass
vehicle_class_stats['Identifier'] = vehicle_class_stats['VehicleClass']
vehicle_class_stats = vehicle_class_stats.drop(columns=['VehicleClass'])

# Flatten the multi-level columns after aggregation
vehicle_name_stats.columns = ['_'.join(col).strip() for col in vehicle_name_stats.columns.values]
vehicle_class_stats.columns = ['_'.join(col).strip() for col in vehicle_class_stats.columns.values]

# Concatenate the two dataframes (VehicleName and VehicleClass)
final_result = pd.concat([vehicle_name_stats, vehicle_class_stats], ignore_index=True)
final_result.set_index('Identifier_', inplace=True)

# Transpose the DataFrame
transposed_result = final_result.transpose()
transposed_result.reset_index()
# Save the result to a CSV with an 'Identifier' column
#transposed_result.to_csv('Vehicle_stats_all_6_scenarios_per_km_160.csv')#, index=False)

# Print the first few rows of the final result to verify
print(transposed_result)

In [ ]:
#costs = pd.read_csv(f"Vehicle_stats_all_6_scenarios_per_km_160.csv")
# Replace the specified scenario name
costs['Unnamed: 0'] = costs['Unnamed: 0'].str.replace('80_100_to_90', '100_80_to_90', regex=False)

# Define scenario names
scenario_names = ['80_90_to_100', '90_100_to_80', '100_80_to_90', '100_to_90', '100_to_110', '100_to_160']

# Extract 'Days_mean' and 'DistanceTravelled_mean' for later use
days_mean = costs[costs['Unnamed: 0'] == 'Days_mean']
distance_travelled = costs[costs['Unnamed: 0'] == 'DistanceTravelled_mean']

# Define crash types that we want to filter on (Fatal, Serious, Minor, Non-injury)
crash_types = ['Fatal', 'Serious', 'Minor', 'Non-injury']

# List to store new rows to append
new_rows = []

# Iterate through each scenario and process the data
for scenario in scenario_names:
    # Filter the rows for the current scenario
    almost2 = costs[costs['Unnamed: 0'].str.contains(scenario, na=False)]
    
    # Filter the rows where 'Unnamed: 0' contains any of the crash types and ends with '_mean'
    crash_rows = almost2[almost2['Unnamed: 0'].str.contains(r'(?:' + '|'.join(crash_types) + r').*_mean$', na=False, regex=True)]
    
    # Sum the relevant columns for these crash types
    crash_sum = crash_rows.drop(columns=['Unnamed: 0']).sum()  # Sum all columns except 'Unnamed: 0'
    
    # Get the 'Days_mean' and 'DistanceTravelled_mean' for the current scenario
    days_value = days_mean.drop(columns=['Unnamed: 0']).iloc[0]  # Assuming one row for Days_mean
    distance_value = distance_travelled.drop(columns=['Unnamed: 0']).iloc[0]  # Assuming one row for DistanceTravelled_mean
    
    # Perform the calculation: (Crash sum * Days_mean) / DistanceTravelled_mean
    crashes_per_km_mean = (crash_sum * days_value) / distance_value
    
    # Name the result for clarity
    crashes_per_km_mean.name = f'Crashes_{scenario}_per_km_mean'
    
    # Add a row for the current scenario's results
    crashes_per_km_mean_row = pd.DataFrame(crashes_per_km_mean).T  # Convert Series to DataFrame and transpose it
    
    # Set the 'Unnamed: 0' column to the corresponding name
    crashes_per_km_mean_row['Unnamed: 0'] = f'Crashes_{scenario}_per_km_mean'
    
    # Append this new row to the list of new rows
    new_rows.append(crashes_per_km_mean_row)
    
    # ** New step: Sum 'Fuel', 'Crashes', and 'Time' rows to create 'Total_{scenario}_per_km_mean' **
    fuel_rows = almost2[almost2['Unnamed: 0'].str.contains('Fuel.*_mean', na=False)]
    crashes_rows = almost2[almost2['Unnamed: 0'].str.contains('Crashes.*_mean', na=False)]
    time_rows = almost2[almost2['Unnamed: 0'].str.contains('Time.*_mean', na=False)]
    
    # Sum the values for Fuel, Crashes, and Time rows
    total_sum = fuel_rows.drop(columns=['Unnamed: 0']).sum() + crashes_rows.drop(columns=['Unnamed: 0']).sum() + time_rows.drop(columns=['Unnamed: 0']).sum()
    
    # Perform the calculation: (Total sum * Days_mean) / DistanceTravelled_mean
    total_per_km_mean = (total_sum * days_value) / distance_value
    
    # Name the result for clarity
    total_per_km_mean.name = f'Total_{scenario}_per_km_mean'
    
    # Add a row for the current scenario's total results
    total_per_km_mean_row = pd.DataFrame(total_per_km_mean).T  # Convert Series to DataFrame and transpose it
    
    # Set the 'Unnamed: 0' column to the corresponding name
    total_per_km_mean_row['Unnamed: 0'] = f'Total_{scenario}_per_km_mean'
    
    # Append this new row to the list of new rows
    new_rows.append(total_per_km_mean_row)

# Concatenate the new rows with the original costs dataframe
costs = pd.concat([costs] + new_rows, ignore_index=True)

# Print the modified costs DataFrame
print(costs)
#costs.to_csv(f"Vehicle_stats_all_6_scenarios_per_km_160.csv")


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your dataframe is named 'costs'
scenario_names = ['80_90_to_100', '90_100_to_80', '100_80_to_90', '100_to_90', '100_to_110', '100_to_160']

# Loop through each scenario name and create a new dataframe for each
for scenario in scenario_names:
    # Filter rows where the scenario name is in the 'Unnamed: 0' column
    scenario_df = costs[costs['Unnamed: 0'].str.contains(scenario, na=False)]
    #print(scenario_df)
    # Select rows where 'Unnamed: 0' starts with 'Time', 'Fuel', or 'Crashes' and ends with '_mean'
    filtered_df = scenario_df[scenario_df['Unnamed: 0'].str.startswith(('Time', 'Fuel', 'Crashes', 'Total')) & scenario_df['Unnamed: 0'].str.endswith('_mean')]
    print(filtered_df)
    # Extract the relevant columns for plotting
    plot_df = filtered_df[['Unnamed: 0', 'SUV', 'Car', 'Truck']]#.set_index('Unnamed: 0')


    order = ['Time','Fuel', 'Crashes', 'Total']

    # Create a sorting key based on the desired order
    plot_df['sort_key'] = plot_df['Unnamed: 0'].apply(lambda x: next((i for i, word in enumerate(order) if x.startswith(word)), float('inf')))
    
    # Sort the DataFrame by the sorting key and drop the key afterward
    plot_df = plot_df.sort_values('sort_key').drop(columns='sort_key').reset_index(drop=True)
    plot_df = plot_df.set_index('Unnamed: 0')

        # Create a bar chart for each scenario
    plot_df.T.plot(kind='bar', width=0.5, zorder=2)

    # Set the title and labels
    plt.title(f'Posted Speed Limit Change of {scenario}')
    plt.xlabel('Category')
    plt.ylabel('Cost Change [$/km]')
    plt.xticks(rotation=0)
    plt.grid(True, zorder=1)

    # Set the x-axis labels to be the category names: SUV, Car, Truck
    plt.xticks(range(3), ['SUV', 'Car', 'Truck'])
    plt.legend(['Time','Fuel', 'Crashes', 'Total'], title='Metrics')

    # Show the plot
    plt.tight_layout()  # Adjust layout to avoid clipping

    # Optionally save the plot if needed
    #plt.savefig(f"{scenario}_category_comparison.png")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd

# Assuming your dataframe is named 'costs'
scenario_names = ['80_90_to_100', '90_100_to_80', '100_80_to_90', '100_to_90', '100_to_110', '100_to_160']

# Create the figure and a custom gridspec layout
#spec = gridspec.GridSpec(nrows=2, ncols=3, figure=fig, height_ratios=[1, 1], hspace=0.4)
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))  # Use subplots to generate axes
axes = axes.flatten() 

# Loop through each scenario name and create a new dataframe for each
for i, scenario in enumerate(scenario_names):
    ax = axes[i]
    # Filter rows where the scenario name is in the 'Unnamed: 0' column
    scenario_df = costs[costs['Unnamed: 0'].str.contains(scenario, na=False)]
    #print(scenario_df)
    # Select rows where 'Unnamed: 0' starts with 'Time', 'Fuel', 'Crashes', or 'Total' and ends with '_mean'
    filtered_df = scenario_df[
        scenario_df['Unnamed: 0'].str.startswith(('Time', 'Fuel', 'Crashes', 'Total')) & 
        scenario_df['Unnamed: 0'].str.endswith('_mean')]
    print(filtered_df)
    # Extract the relevant columns for plotting
    plot_df = filtered_df[['Unnamed: 0', 'SUV', 'Car', 'Truck']]

    order = ['Time', 'Fuel', 'DiffCrashes', 'Total']

    # Create a sorting key based on the desired order
    plot_df['sort_key'] = plot_df['Unnamed: 0'].apply(lambda x: next((i for i, word in enumerate(order) if x.startswith(word)), float('inf')))
    
    # Sort the DataFrame by the sorting key and drop the key afterward
    plot_df = plot_df.sort_values('sort_key').drop(columns='sort_key').reset_index(drop=True)
    plot_df = plot_df.set_index('Unnamed: 0')

    # Create a bar chart for the current scenario
    ax = plot_df.T.plot(kind='bar', width=0.5, ax=axes[i], zorder=2, legend=False)
    
    # Set the title and labels
    ax.set_title(f'{scenario}')
    ax.set_xticks(range(3))
    ax.set_xticklabels(['SUV', 'Car', 'Truck'], rotation=0)
    ax.grid(True, zorder=1)

# Set the shared y-label and legend
fig.text(0.04, 0.5, '$/km', va='center', rotation='vertical', fontsize='large')
fig.legend(['Time', 'Fuel', 'Crashes', 'Total'], title='Metrics', loc='upper center', ncol=4, bbox_to_anchor=(0.5, 1.05))

# Adjust layout to avoid overlap
plt.subplots_adjust(left=-0.1, right=1, top=0.9, bottom=0.1, wspace=0.3, hspace=0.5)

# Save and show the combined plot
#plt.savefig(f'Combined_graph_of_cost_per_km_5_figures.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your dataframe is named 'costs'
scenario_names = ['80_90_to_100', '90_100_to_80', '100_80_to_90', '100_to_90', '100_to_110', '100_to_160']

# Create the figure and subplots
# Create the figure and a custom gridspec layout
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))  # Use subplots to generate axes
axes = axes.flatten() 

# Loop through each scenario name and create a new dataframe for each
for i, scenario in enumerate(scenario_names):
    # Filter rows where the scenario name is in the 'Unnamed: 0' column
    scenario_df = costs[costs['Unnamed: 0'].str.contains(scenario, na=False)]
    
    # Select rows where 'Unnamed: 0' starts with 'Fuel' or 'Crashes' and ends with '_mean'
    filtered_df = scenario_df[
        scenario_df['Unnamed: 0'].str.startswith(('Fuel', 'Crashes')) & 
        scenario_df['Unnamed: 0'].str.endswith('_mean')
    ]
    
    # Pivot the DataFrame to get the data in a plottable format
    plot_df = filtered_df[['Unnamed: 0', 'SUV', 'Car', 'Truck']].set_index('Unnamed: 0')
    
    sum_row = plot_df.sum(axis=0).to_frame().T  # Sum along rows (column-wise)
    sum_row.name = 'Total'  # Optional: Give the new row a label
    plot_df = pd.concat([plot_df,sum_row])

    # Calculate the "Total" as the sum of "Fuel" and "Crashes"
     # Create a bar chart for the current scenario
    ax = plot_df.T.plot(kind='bar', width=0.5, ax=axes[i],color=['tab:orange','tab:green','tab:red'], zorder=2, legend=False)
    
    # Plot the data for the current scenari
        # Set the title and labels
    ax.set_title(f'{scenario}')
    #axes[i].set_xlabel('Category')
    ax.set_xticks(range(3))
    ax.set_xticklabels(['SUV', 'Car', 'Truck'], rotation=0)
    ax.grid(True, zorder=1)

# Set the shared y-label and legend
fig.text(0.04, 0.5, '$/km', va='center', rotation='vertical', fontsize='large', position=(0.004, 0.5))
fig.legend(['Fuel', 'Crashes', 'Total'], title='Metrics', loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.05))

# Adjust layout to avoid overlap
plt.subplots_adjust(left=-0.1, right=1, top=0.9, bottom=0.1, wspace=0.3, hspace=0.5)
#fig.savefig(f'Combined_graph_No_Time_of_cost_per_km_160.png')
# Show the combined plot
plt.show()

In [ ]:
#almost2 = pd.read_csv(f"Cost_per_FuelUp_All_Vehicles_All_Models_160.csv")
#print(almost2)

almost2['kms/day'] = almost2['DistanceTravelled'] / almost2['Days']
#print(almost2['kms/day'])
for unique in almost2['VehicleName'].unique():
    Unique = almost2[almost2['VehicleName']==unique].copy()
    mean = Unique['kms/day'].mean()
    std = Unique['kms/day'].std()
    #print(f"Vehicle {unique} travelled {mean:.2f}±{std:.2f}km/day")

for unique in almost2['VehicleClass'].unique():
    Unique = almost2[almost2['VehicleClass']==unique].copy()
    mean = Unique['kms/day'].mean()
    std = Unique['kms/day'].std()
    #print(f"Vehicle Class {unique} travelled {mean:.2f}±{std:.2f}km/day")

#almost1 = pd.read_csv(f"Vehicle_stats_all_6_scenarios_per_km_160.csv")
almost1 = almost1[['Unnamed: 0', 'SUV', 'Car', 'Truck']]
#print(almost1)
scenario_names = ['80_90_to_100', '90_100_to_80', '100_80_to_90', '100_to_90', '100_to_110', '100_to_160']
filtered_dfs = []

for scenario in scenario_names:
    almost2 = almost1[almost1['Unnamed: 0'].str.contains(scenario, na=False)]
    # Filter rows where 'Unnamed: 0' ends with '_mean' and starts with one of the desired prefixes
    filtered_df = almost2[
        almost2['Unnamed: 0'].str.endswith('_mean') & 
        almost2['Unnamed: 0'].str.startswith(('Fuel', 'Crashes', 'Time', 'Total'))
    ].copy()  # Using .copy() to avoid setting a value on a view
    
    # Add the 'Scenario' column with the current scenario
    filtered_df['Scenario'] = scenario
    
    # Add the 'Phrase' column, which extracts the prefix (Fuel, Crashes, Time, Total) from 'Unnamed: 0'
    filtered_df['Phrase'] = filtered_df['Unnamed: 0'].str.extract(r'^(Fuel|Crashes|Time|Total)')
    
    # Append the filtered DataFrame to the list
    filtered_dfs.append(filtered_df)

# Concatenate all filtered DataFrames into one
final_df = pd.concat(filtered_dfs, ignore_index=True)
final_df = final_df.drop(columns=['Unnamed: 0'])
final_df[['SUV ($/km)','Car ($/km)','Truck ($/km)']] = final_df[['SUV','Car','Truck']]
final_df = final_df[['Scenario', 'Phrase', 'SUV ($/km)', 'Car ($/km)', 'Truck ($/km)']]
# Print the resulting DataFrame
print(final_df)
#final_df.to_csv(f'combined_totals_with_time_160.csv')

In [ ]:
import pandas as pd

# Fleet composition
num_suvs = 18
num_cars = 2
num_trucks = 1

#Car dominated fleet 
#num_suvs = 2
#num_cars = 18
#num_trucks = 1

# Average daily distances and their bounds
avg_suv_dist_per_day = 73.7
avg_suv_dist_per_day_lower = 43.3

avg_truck_dist_per_day = 30.1
avg_truck_dist_per_day_lower = 26.2

avg_car_dist_per_day = 75.8
avg_car_dist_per_day_lower = 36.7

# Read cost data
cost_per_km = pd.read_csv('combined_totals_with_time_160.csv')
print(cost_per_km)

# Calculate daily costs for each vehicle type (assuming there's a column 'cost_per_km' in the dataframe)
cost_per_km['Cost_per_day_suv'] = cost_per_km['SUV ($/km)'] * avg_suv_dist_per_day * num_suvs
cost_per_km['Cost_per_day_car'] = cost_per_km['Car ($/km)'] * avg_car_dist_per_day * num_cars
cost_per_km['Cost_per_day_truck'] = cost_per_km['Truck ($/km)'] * avg_truck_dist_per_day * num_trucks
cost_per_km['Cost_per_day_fleet'] = cost_per_km['Cost_per_day_suv']+cost_per_km['Cost_per_day_car']+cost_per_km['Cost_per_day_truck']
cost_per_km['Fleet_daily_distance'] = avg_suv_dist_per_day * num_suvs + avg_truck_dist_per_day * num_trucks + avg_car_dist_per_day * num_cars

# Calculate daily costs based on lower bounds
cost_per_km['Cost_per_day_suv_std'] = cost_per_km['SUV ($/km)'] * avg_suv_dist_per_day_lower * num_suvs
cost_per_km['Cost_per_day_car_std'] = cost_per_km['Car ($/km)'] * avg_car_dist_per_day_lower * num_cars
cost_per_km['Cost_per_day_truck_std'] = cost_per_km['Truck ($/km)'] * avg_truck_dist_per_day_lower * num_trucks
cost_per_km['Cost_per_day_fleet_std'] = cost_per_km['Cost_per_day_suv_std']+cost_per_km['Cost_per_day_car_std']+cost_per_km['Cost_per_day_truck_std']

days_per_year = 365.25
cost_per_km['Cost_per_year_fleet'] = cost_per_km['Cost_per_day_fleet'] * days_per_year
cost_per_km['Cost_per_year_fleet_std'] = cost_per_km['Cost_per_day_fleet_std'] * days_per_year
cost_per_km['Fleet_yearly_distance'] = cost_per_km['Fleet_daily_distance']*days_per_year

print(cost_per_km)
#cost_per_km.to_csv(f'combined_totals_with_time_160_all_cars_fleet.csv')

In [ ]:
cost_per_km['Litre_change_per_year_fleet'] = np.nan
cost_per_km['Hour_change_per_year_fleet'] = np.nan
# Apply the calculation conditionally for rows where Phrase is 'Fuel'
cost_per_km.loc[cost_per_km['Phrase'] == 'Fuel', 'Litre_change_per_year_fleet'] = (cost_per_km['Cost_per_year_fleet'] / 2.189)
cost_per_km.loc[cost_per_km['Phrase'] == 'Time', 'Hour_change_per_year_fleet'] = (cost_per_km['Cost_per_year_fleet'] / 150)
cost_per_km.loc[cost_per_km['Phrase'] == 'Fuel', 'CO2_emission_change_per_year_fleet'] = (2.68*cost_per_km['Litre_change_per_year_fleet'])
print(cost_per_km)
#cost_per_km.to_csv(f'combined_totals_with_time_160_all_cars_fleet.csv')

# Find percentage difference of fuel and time in each model

In [ ]:
#df = pd.read_csv(f'AAA_All_Vehicles_All_Model_160.csv')
df['TrueFuel'] = df['FuelModel']
df.drop(columns=['FuelModel'])
print(df.columns)

In [ ]:
time_sums = df.filter(like='Time').sum()
print("Time Sums:")
print(time_sums)

# Summing up 'Fuel' columns
fuel_sums = df.filter(like='Fuel').sum()
print("\nFuel Sums:")
print(fuel_sums)

In [ ]:
time_sums = df.filter(like='Time').sum()
time_percentage_diff = ((time_sums - time_sums['TrueTime']) / time_sums['TrueTime']) * 100

# Summing up 'Fuel' columns and computing percentage difference relative to 'TrueFuel'
fuel_sums = df.filter(like='Fuel').sum()
fuel_percentage_diff = ((fuel_sums - fuel_sums['TrueFuel']) / fuel_sums['TrueFuel']) * 100

# Printing results
print("Time Percentage Differences:")
print(time_percentage_diff)

print("\nFuel Percentage Differences:")
print(fuel_percentage_diff)